## Shark Attacks


In [16]:
import pandas as pd
import re
data = pd.read_csv("input/GSAF5.csv", encoding = "ISO-8859-15")

In [17]:
data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


### 1st clean:
Columns 'Unnamed: 22' & 'Unnamed: 23' are not referenced in the description of the dataset and doesn't contain any relevant information. Proceed to drop them.

In [18]:
data = data.drop(['Unnamed: 22','Unnamed: 23'], axis=1)

### 2nd clean: Renaming columns
- Some names of the columns aren't clean or clear enough. Below the list of columns renamed
    - Sex: remove a blank space at the end.

In [19]:
data.rename(columns={'Sex ':'Sex'}, inplace=True)

### 3rd clean: Change column types

In [20]:
data.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
dtype: object

### 4th clean: 

Among the total 5900 events registered, only 137 happened before 1700. To evaluate only statistically relevant data, events registered before 1700 will not be considered

In [21]:
data = data[data['Year'] > 1700]

In [23]:
data['Sex'].value_counts()

M      4721
F       572
M         2
.         1
N         1
lli       1
Name: Sex, dtype: int64

In [22]:
display(data[(data['Sex'] == 'M ')])

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
341,2014.04.22,22-Apr-14,2014,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Swimming,male,M,...,N,15h30,NaN,"R. Neale, Florida Today, 4/22/2014",2014.04.22-CocoaBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2014.04.22,2014.04.22,5652
1363,2005.05.15,15-May-05,2005,Unprovoked,AUSTRALIA,Queensland,50 km east of Townsville,Spearfishing,Ben Edelstein,M,...,N,NaN,Blacktip shark,"J. Anderson, Townsville Bulletin, 5/21/2005",2005.05.15-Edelstein.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2005.05.15,2005.05.15,4630


In [8]:
data['Sex'].rename({341:'M', 1363:'M'}, inplace=True)

In [24]:
null_cols = data.isnull().sum()
print(null_cols[null_cols > 0])
print(data.shape)


Country                     36
Area                       370
Location                   453
Activity                   502
Name                       193
Sex                        554
Age                       2554
Injury                      21
Fatal (Y/N)                 19
Time                      3080
Species                   2830
Investigator or Source      15
href formula                 1
href                         3
dtype: int64
(5852, 22)


In [11]:
data.shape

(5852, 22)

In [25]:
# Regular Expressions: Clean Country, Extract month

In [58]:
hour = [data.loc[:,'Time'] != 'NaN']

In [50]:
set(data['Time'])

{' ',
 '  ',
 '   ',
 ' 14h00',
 '"After dark"',
 '"After lunch"',
 '"Early evening"',
 '"Evening"',
 '"Just before 11h00"',
 '"Night"',
 '"shortly before dusk"',
 '--',
 '00h30',
 '01h00',
 '01h30',
 '01h32',
 '01h50',
 '02h00',
 '02h30',
 '02h45',
 '03h00',
 '03h30',
 '03h45 - 04h00',
 '04h00',
 '05h00',
 '05h30',
 '05h40',
 '05h45',
 '05h50',
 '06h00',
 '06h00 -- 07h00',
 '06h10',
 '06h15',
 '06h30',
 '06h45',
 '06h47',
 '06h50',
 '06j00',
 '07h00',
 '07h00 - 08h00',
 '07h05',
 '07h08',
 '07h10',
 '07h15',
 '07h19',
 '07h20',
 '07h30',
 '07h32',
 '07h40',
 '07h45',
 '07h50',
 '07h56',
 '08h00',
 '08h00 / 09h30',
 '08h05',
 '08h10',
 '08h15',
 '08h20',
 '08h30',
 '08h35',
 '08h40',
 '08h45',
 '08h50',
 '08h55',
 '08h57',
 '08h58',
 '09h00',
 '09h00 - 09h30',
 '09h00 -10h00',
 '09h05',
 '09h11',
 '09h15',
 '09h20',
 '09h30',
 '09h30 ',
 '09h30 / 10h00',
 '09h30 / 15h30',
 '09h35',
 '09h40',
 '09h45',
 '09h50',
 '09h55',
 '09h57',
 '10h00',
 '10h00 -- 11h00',
 '10h00 / 11h00',
 '10h00 